Welcome to the **[30 Days of ML competition](https://www.kaggle.com/c/30-days-of-ml/overview)**!  In this notebook, you'll learn how to make your first submission.

Before getting started, make your own editable copy of this notebook by clicking on the **Copy and Edit** button.

# Step 1: Import helpful libraries

We begin by importing the libraries we'll need.  Some of them will be familiar from the **[Intro to Machine Learning](https://www.kaggle.com/learn/intro-to-machine-learning)** course and the **[Intermediate Machine Learning](https://www.kaggle.com/learn/intermediate-machine-learning)** course.

In [ ]:
# Familiar imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import os
import glob
from datetime import datetime
from pathlib import Path

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from scipy.special import boxcox1p



# For ordinal encoding categorical variables, splitting data, pipeline, and so on
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, PowerTransformer, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold, cross_val_score 
from sklearn.pipeline import make_pipeline

# For training random forest model
from sklearn.linear_model import ElasticNet, Lasso
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

# base
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin, clone


from sklearn.metrics import mean_squared_error


In [ ]:
pd.set_option("display.max_columns", 100)

# Step 2: Load the data

Next, we'll load the training and test data.  

We set `index_col=0` in the code cell below to use the `id` column to index the DataFrame.  (*If you're not sure how this works, try temporarily removing `index_col=0` and see how it changes the result.*)

In [ ]:
# Load the training data
train = pd.read_csv("train.csv", index_col=0)
test = pd.read_csv("test.csv", index_col=0)

# Preview the data
train.head()
train.describe().T

The next code cell separates the target (which we assign to `y`) from the training features (which we assign to `features`).

In [ ]:
# Separate target from features
y = train['target']
features = train.drop(['target'], axis=1)

# Preview features
features.head().T

# Step 3: Prepare the data

Next, we'll need to handle the categorical columns (`cat0`, `cat1`, ... `cat9`).  

In the **[Categorical Variables lesson](https://www.kaggle.com/alexisbcook/categorical-variables)** in the Intermediate Machine Learning course, you learned several different ways to encode categorical variables in a dataset.  In this notebook, we'll use ordinal encoding and save our encoded features as new variables `X` and `X_test`.

In [ ]:
def add_poly_features(train, test, cols=None, concate=True, poly_degree=2):
    
    if cols is  None:
        cols = train.columns
     
    columns = train[cols].columns
    poly = PolynomialFeatures(poly_degree)
    poly_train = pd.DataFrame(poly.fit_transform(train[cols]), index=train.index)
    poly_test = pd.DataFrame(poly.transform(test[cols]), index=test.index)
    
    # stamp these columns
    poly_train = poly_train.add_prefix('poly_')
    poly_test = poly_test.add_prefix('poly_')
    
    if concate:
        train = pd.concat([train, poly_train], axis=1)
        test = pd.concat([test, poly_test], axis=1)
        
    return train, test



In [ ]:

# List of categorical columns
object_cols_long = [col for col in features.columns if 'cat' in col]
cont_cols = [col for col in features.columns if 'con' in col]

X = features.copy()
X_test = test.copy()

# cols = ['cont0', 'cont3', 'cont9']
# add poly features
# X, X_test = add_poly_features(X, X_test, cols)




Next, we break off a validation set from the training data.

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=0)

In [ ]:
X_train.shape

### Transformers

In [ ]:
# transfomration: scaling, encoding, new features, ... etc
# helpers
# map columns to indexes 
def get_columns
class ML30Transformer(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 cat_columns=None,
                 cont_columns=None,
                 target_column=None,
                 scaler=StandardScaler(),
                 normalizer=None,
                 cont_encoder=OrdinalEncoder()
                 target_encoder=None):
        
        self.cat_columns = cat_columns
        self.cont_columns = cont_columns
        self.target_column = target_column
        self.scaler = scaler
        self.normalizer = normalizer
        self.cont_encoder = cont_encoder
    
    def fit(X):
        
        self.cont_encoder.fit(X[self.cont_columns])
        return self
    
    def transform(X):
        
        return self.cont_encoder.transform(X[self.cont_columns])
        
        
        
        

In [ ]:
# encoding
#ml30_transformer = ML30Transformer(cont_columns=X.columns.get_indexer(cont_cols))
#X[object_cols_long] = ml30_transformer.fit_transform(X[object_cols_long])
#X_test[object_cols_long] = ml30_transformer.transform(X_test[object_cols_long])

# Step 4.1: Predict using Generalized Stacking


In [ ]:

# helpers and wrappers

## helper fucntions
def store_oof_predictions(model_name,
                          valid_predictions,
                          test_predictions,
                          X_train,
                          X_test,
                          stacking_level,
                          output_folder,
                          n_folds, 
                          random_state):    
    
    """
    Store oof predictions into files.
    It is safe not to use defaults for this helper to make sure 
    we are storing the right experiement. 
    """
    
    if test_predictions is not None:
        test_df = pd.DataFrame({'Id': X_test.index,
                       model_name: test_predictions})
        test_df.to_csv( f'{output_folder}{os.sep}{model_name}_{n_folds}_test.csv',
                       index=False)

    if valid_predictions is not None:
        valid_df = pd.DataFrame({'Id': X_train.index,
                           model_name: valid_predictions})
        valid_df.to_csv(f'{output_folder}{os.sep}{model_name}_{n_folds}_valid.csv', 
                        index=False)
    
    
def read_oof_predictions(pattern, 
                     models_names, 
                     index='id'):    
    """
    Read files according the pattern in their name
    it returns a dataframe where each column is the prediction of a model
    """
    li = [] 
    for f in glob.glob(pattern):
        file_name = f.split(os.sep)[-1] # get file name only
        print(file_name)
        if file_name.lower().startswith(tuple(s.lower() for s in models_names)): # get only required models
            df = pd.read_csv(f, index_col=0)
            li.append(df)
    
                                   
    return pd.concat(li, axis=1)
    

In [ ]:
class ModelWrapper():
    def __init__(self, 
                 model,
                 name,
                 uses_eval_set=False,
                 fit_params={}):
        
        self.model = model
        self.name = name
        
        self.uses_eval_set = uses_eval_set
        self.fit_params = fit_params # any extra params for the 'fit' function
        
#         self.valid_predictions = None # oof predictions on the valiation set
#         self.test_predictions = None  # meta-features on the test set
                
    def fit(self, X, y, eval_set=None):
        if self.uses_eval_set:
            self.model.fit(X, y, eval_set=eval_set, **(self.fit_params))
        else:
            self.model.fit(X, y, **(self.fit_params)) 
        return self
    

    def predict(self, X):
        return self.model.predict(X)
        
    def clone_me(self, random_state=None):
        wrapper = ModelWrapper(model=clone(self.model), # clone from sklean.base
                               name=self.name, 
                               uses_eval_set=self.uses_eval_set,
                               fit_params=self.fit_params)
        wrapper.name = self.name
        if random_state is not None:
            wrapper.set_random_state(random_state)
        
        return wrapper
    
    def set_random_state(self, random_state):
        if hasattr(self.model, 'random_state'):
            self.model.random_state = random_state
        elif hasattr(self.model, 'random_seed'):
            self.model.random_seed = random_state
            
    def get_random_state(self):
        if hasattr(self.model, 'random_state'):
            return self.model.random_state 
        elif hasattr(self.model, 'random_seed'):
            return self.model.random_seed
    
        

In [442]:
class ModelTrainer():
    def __init__(self,
                  model: ModelWrapper):
        
        self.model = model
        
    def calc_oofs(self,
                  X, y,
                  X_test,
                  transformer,
                  folds,
                  verbose=False,
                  use_different_random_states=True):
        """
        Return the oofs predictions and the meta features (test predictions)
        """
        
        test_predictions = 0
        oof_predictions = np.zeros_like(np.array(y))
        valid_mean_score = [] 
                
        for fold, (train_ix, valid_ix) in enumerate(folds.split(X)):
            X_train, X_valid = X[train_ix], X[valid_ix]
            y_train, y_valid = y[train_ix], y[valid_ix]
            
            # transform input
            X_train = transformer.fit_transform(X_train)
            X_valid = transformer.transform(X_valid)
            
            
            # check if we train each fold on differently initialized clone
            if use_different_random_states:
                model = self.model.clone_me(random_state=fold)
            else:
                model = self.model.clone_me()
            
            # fit the model
            if model.uses_eval_set:
                model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)])
            else:
                model.fit(X_train, y_train)
                
            ## predictions
            # on the validation set
            valid_predications = model.predict(X_valid)
            score = mean_squared_error(valid_predications, y_valid, squared=False)
            valid_mean_score.append(score)
            oof_predictions[valid_ix] = valid_predications
            
            # on the test set
            # transform it first on a copy
            X_test_ = transformer.transform(X_test)
            test_predictions += model.predict(X_test_) / folds.n_splits
            
            if verbose:
                print('Fold:{} score:{:.4f}'.format(fold + 1, score))
        
        if verbose:
            print('Average score:{:.4f} ({:.4f})'.format(np.mean(valid_mean_score), np.std(valid_mean_score) ))
    
        return oof_predictions, test_predictions

In [449]:

class Experiment():
    def __init__(self,
                 title='Experiment',
                 n_folds=5,
                 random_state=42,
                 shuffle=True,
                 use_different_random_states=True,
                 main_folder=os.getcwd()):
        
        self.title = title
        self.n_folds = n_folds
        self.random_state = random_state
        self.shuffle = shuffle
        self.use_different_random_states = use_different_random_states
        self.main_folder = main_folder
        
        

    def create(self, use_folder=None, resume=False):
        
        # create folds
        self.folds = KFold(n_splits=self.n_folds, 
                           random_state=self.random_state,
                           shuffle=self.shuffle)
        
        
        # create time stamp and subfolder with the current time stamp
        if use_folder is not None:
            self.output_folder = use_folder 
        else:
            time_stamp = datetime.now().isoformat(' ', 'seconds')
            self.output_folder = self.title + ' ' + time_stamp.replace(':', '-')
            Path(f'{self.main_folder}{os.sep}{self.output_folder}').mkdir(parents=True, exist_ok=True)
        
     
    def to_dict(self):
        
        return dict({"title": self.title,
                    "n_folds": self.n_folds,
                    "random_state": self.random_state,
                    "use_different_random_states": self.use_different_random_states,
                    "main_folder": self.main_folder,
                    "output_folder": self.output_folder})
        
    def __str__(self):
        
        return str(self.to_dict())

In [ ]:
class LevelTrainer():
    
    def __init__(self,
                 experiment, # Experiment,
                 models,      # ModelWrapper
                 transformer, # data transformer
                 level_num=1):
        
        self.level_num = level_num
        self.experiment = experiment
        self.models = models
        
    def train(self, X_df, y_df, X_test_df, verbose=True):
        
        assert models is not None
        

        for model_wrapper in self.models:
            
            if verbose:
                print('-'*3)
                print(f'Model:{model_wrapper.name}')
                print('-'*3)
                
            trainer = ModelTrainer(model_wrapper)
            oof_preds, test_preds = trainer.calc_oofs(X.values, y.values,
                                                      X_test.values,
                                                      transformer=self.transformer,
                                                      folds=self.experiment.folds,
                                                      verbose=verbose,
                                                      use_different_random_states=self.experiment.use_different_random_states)
            # store predictions
            store_oof_predictions(model_name=model_wrapper.name,
                                  valid_predictions=oof_preds,
                                  test_predictions=test_preds,
                                  X_train=X_df,
                                  X_test=X_test_df,
                                  stacking_level=self.level_num, 
                                  n_folds=self.experiment.n_folds, 
                                  random_state=model_wrapper.get_random_state(),
                                  output_folder=f'{self.experiment.main_folder}{os.sep}{self.experiment.output_folder}')
            if verbose:
                print('-'*3)

### ML Models

#### Hyperparamater ranges
common ranges (to be added)

#### Hyperparameters: these should go to a JSON file

In [ ]:
# extra-tree
et_params = {
    'n_jobs': -1,
    'n_estimators': 100,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf': 2,
}

# random forest
rf_params = {
    'n_jobs': -1,
    'n_estimators': 100,
    'max_features': 0.2,
    'max_depth': 8,
    'min_samples_leaf': 2,
}

# xgboost
# xgb_params = {'n_estimators': 10000,
#               'learning_rate': 0.35,
#               'subsample': 0.926,
#               'colsample_bytree': 0.84,
#               'max_depth': 2,
#               'booster': 'gbtree', 
#               'reg_lambda': 35.1,
#               'reg_alpha': 34.9,
#               'random_state': 42,
#               'n_jobs': -1}

# Using optuna but still rough
xgb_params = {'max_depth': 5,
             'learning_rate': 0.021252439960137114,
             'n_estimators': 13500,
             'subsample': 0.62,
             'booster': 'gbtree',
             'colsample_bytree': 0.1,
             'reg_lambda': 0.1584605320779582,
             'reg_alpha': 15.715145781076245,
             'n_jobs': -1}

# catboost
catb_params = {'iterations': 6800,
              'learning_rate': 0.93,
              'loss_function': "RMSE",
              'random_state': 42,
              'verbose': 0,
              'thread_count': 4,
              'depth': 1,
              'l2_leaf_reg': 3.28}

SEED = 7770777
params_lgb = {"n_estimators": 10000,   
            "boosting_type": "gbdt",
            "objective": "regression",
            "metric": "rmse",
            "learning_rate": 0.007899156646724397,
            "num_leaves": 77,
            "max_depth": 77,
            "feature_fraction": 0.2256038826485174,
            "bagging_fraction": 0.7705303688019942,
            "min_child_samples": 290,
            "reg_alpha": 9.562925363678952,
            "reg_lambda": 9.355810045480153,
            "max_bin": 772,
            "min_data_per_group": 177,
            "bagging_freq": 1,
            "cat_smooth": 96,
            "cat_l2": 17,
            "verbosity": -1,
            "bagging_seed": SEED,
            "feature_fraction_seed": SEED,
            "verbose_eval":1000,
            "seed": SEED,
            "n_jobs":-1}


In [ ]:
# external hyperparamaters

### fit function hyperparamaters
# some models require special paramaters like early stoping in xgboost and lgbm

# xgboost
xgb_fit_params = {'early_stopping_rounds': 200,
                  'verbose': False}

# lgbm
lgb_fit_params = {'early_stopping_rounds': 200,
                  'verbose': False}

### application/implementation paramaters
# These paramaters are implementation dependent 

# xgboost
xgb_app_params = {'uses_eval_set':True}

# lgbm
lgb_app_params = {'uses_eval_set':True}

#### Models

In [ ]:
# lasso
lasso = Lasso(alpha =0.00005)

# Elastic net
e_net = ElasticNet(alpha=0.00005, l1_ratio=.9)

# KNeighborsRegressor
knn =  KNeighborsRegressor()

# extra-tree
extree = ExtraTreesRegressor(**et_params)

# random forest
rfr = RandomForestRegressor(**rf_params)

# xgboost
xgb =  XGBRegressor(**xgb_params)

#catboost
catb = CatBoostRegressor(**catb_params)

#lgbm
lgb = LGBMRegressor(**params_lgb)


In [444]:
# compile all settings in one dictionary, 
# we can store/load it then to a JSON file
models = {'Lasso': {"model":lasso, "fit_kwargs":None, "app_params": None},
          'ElasticNet': {"model": e_net, "fit_kwargs":None, "app_params": None},
          'ExtraTreesRegressor': {"model": extree, "fit_kwargs":None, "app_params": None},
          'RandomForestRegressor': {"model": rfr, "fit_kwargs":None, "app_params": None},
          'XGBRegressor': {"model": xgb, "fit_kwargs":xgb_fit_params, "app_params": xgb_app_params},
          'CatBoostRegressor': {"model": catb, "fit_kwargs":None, "app_params": None},
          'LGBMRegressor': {"model": lgb, "fit_kwargs":lgb_fit_params, "app_params": lgb_app_params}
        }

In [445]:
models.keys()

dict_keys(['Lasso', 'ElasticNet', 'ExtraTreesRegressor', 'RandomForestRegressor', 'XGBRegressor', 'CatBoostRegressor', 'LGBMRegressor'])

### Stacking

Stacking with two levels:
we need two things:

oof predictions and test results.
- train level 1 models and collect their oof predictions 
- use oof predictions in another model and predict the final target.

- there are some variations to this simple procedure: see 
 - https://mlwave.com/kaggle-ensembling-guide/
 - https://www.kaggle.com/getting-started/18153#post103381
 
- one approach would be to:
    - train N models in the first level and get their oof predictions
    - train a model on the whole train set and get it's predictions
    

In [446]:
# settings: experiment and stacking architecutre



level_1_models = ['CatBoostRegressor', 'LGBMRegressor']
# level_1_models = ['all'] # to select all models

output_path = os.getcwd()

ml30_experiment = Experiment(title='ML 30 days',
                        n_folds=5,
                        random_state=42,
                        shuffle=True,
                        use_different_random_states=True,
                        main_folder=f'{output_path}{os.sep}Experiments')

ml30_experiment.create(use_folder='ML 30 days 2021-08-23 17-02-31', resume=True)

print(ml30_experiment)

stack = { 'level_1_models': level_1_models, 'meta_model': 'XGBRegressor'}


# if level_1_models is set to 'all' use the models
if stack['level_1_models'][0].lower() == 'all':
    level_1_models_names = models.keys()
else: 
    level_1_models_names = stack['level_1_models']

{'title': 'ML 30 days', 'n_folds': 5, 'random_state': 42, 'use_different_random_states': True, 'main_folder': '/home/ftam/Documents/data science/kaggel_git/data_science/Experiments', 'output_folder': 'ML 30 days 2021-08-23 17-02-31'}


In [447]:
# main procedure

# create models
model_wrappers = []

for model_name in level_1_models_names:
    
    # get paramaters  
    model = models[model_name]
    fit_kwargs = models[model_name]['fit_kwargs']
    app_params = models[model_name]['app_params']
    
    model_wrapper = ModelWrapper(model=model['model'], name=model_name)
    if fit_kwargs is not None:
        model_wrapper.fit_params = fit_kwargs
    if app_params is not None:
        model_wrapper.uses_eval_set = app_params['uses_eval_set']
    model_wrappers.append(model_wrapper)


In [448]:
# train the levels
ml30_transformer = ML30Transformer(cont_columns=X.columns.get_indexer(cont_cols))

level_trainer = LevelTrainer(experiment=ml30_experiment, 
                             models=model_wrappers, 
                             transformer=ml30_transformer)

level_trainer.train(X_df=X, y_df=y, X_test_df=X_test)

---
Model:CatBoostRegressor
---
Fold:1 score:0.7176
Fold:2 score:0.7174
Fold:3 score:0.7189
Fold:4 score:0.7193
Fold:5 score:0.7177
Average score:0.7182 (0.0008)
---
---
Model:LGBMRegressor
---
Fold:1 score:0.7169
Fold:2 score:0.7169
Fold:3 score:0.7187
Fold:4 score:0.7185
Fold:5 score:0.7172
Average score:0.7176 (0.0008)
---


In [499]:
                         
# required results                            
fold_id = 5
folder ='Experiments/ML 30 days 2021-08-23 17-02-31'
                            
level_1_models = [
                  'ExtraTreesRegressor',
                  'LGBMRegressor',
                  'CatBoostRegressor',
                  'ElasticNet'
                  ]

meta_regressor = 'CatBoostRegressor'



# new features 
X_train_ = read_oof_predictions(pattern=f"{folder}{os.sep}*{fold_id}_valid.csv",
                           models_names=level_1_models)
X_test_ = read_oof_predictions(pattern=f"{folder}{os.sep}*{fold_id}_test.csv",
                          models_names=level_1_models)

X_train_.head()

ExtraTreesRegressor_5_valid.csv
XGBRegressor_5_valid.csv
LGBMRegressor_5_valid.csv
Lasso_5_valid.csv
RandomForestRegressor_5_valid.csv
ElasticNet_5_valid.csv
CatBoostRegressor_5_valid.csv
RandomForestRegressor_5_test.csv
XGBRegressor_5_test.csv
CatBoostRegressor_5_test.csv
LGBMRegressor_5_test.csv
Lasso_5_test.csv
ElasticNet_5_test.csv
ExtraTreesRegressor_5_test.csv


,ExtraTreesRegressor,LGBMRegressor,ElasticNet,CatBoostRegressor
Id,,,,
1,8.229007,8.432929,8.183710,8.483769
2,8.245625,8.370355,8.477100,8.342437
3,8.147453,8.172136,8.180759,8.251786
4,8.290102,8.362367,8.272037,8.388899
6,8.252074,8.205220,8.317998,8.275428


In [500]:
X_train_.corr()

,ExtraTreesRegressor,LGBMRegressor,ElasticNet,CatBoostRegressor
ExtraTreesRegressor,1.000000,0.659918,0.745362,0.635145
LGBMRegressor,0.659918,1.000000,0.515123,0.954623
ElasticNet,0.745362,0.515123,1.000000,0.514060
CatBoostRegressor,0.635145,0.954623,0.514060,1.000000


In [ ]:
meta_model = ModelWrapper(xgb, name='Meta')
model_trainer = ModelTrainer(meta_model)

meta_valid_results, meta_test_results = model_trainer.calc_oofs(X_train_.values, 
                                                                y.values, 
                                                                X_test_.values, 
                                                                experiment.folds, 
                                                                verbose=True,
                                                                use_different_random_states=True)

In [ ]:
#results = np.exp((np.log(lasso_test_results) + np.log(xgb_test_results) + np.log(lgb_test_results) + np.log(catb_test_results)) / 4 )
results = np.exp(np.sum(np.log(np.column_stack(test_stack)) / 4, axis=1))
#mean_squared_error(y, y3, squared=False)

In [493]:
predictions = meta_test_results

<hr>

In the code cell above, we set `squared=False` to get the root mean squared error (RMSE) on the validation data.

# Step 5: Submit to the competition

We'll begin by using the trained model to generate predictions, which we'll save to a CSV file.

In [494]:
predictions.shape

(200000,)

In [495]:
X_test.index

Int64Index([     0,      5,     15,     16,     17,     19,     20,     21,
                23,     29,
            ...
            499981, 499982, 499983, 499984, 499986, 499987, 499990, 499991,
            499994, 499995],
           dtype='int64', name='id', length=200000)

In [496]:
# Save the predictions to a CSV file
output = pd.DataFrame({'Id': X_test.index,
                       'target': predictions})
output.to_csv('submission.csv', index=False)

In [497]:
output.head(10)

,Id,target
0,0,8.088670
1,5,8.378148
2,15,8.420147
3,16,8.425104
4,17,8.172484
5,19,8.275748
6,20,8.395084
7,21,7.828260
8,23,8.129872
9,29,8.330977


In [498]:
output.head(10)

,Id,target
0,0,8.088670
1,5,8.378148
2,15,8.420147
3,16,8.425104
4,17,8.172484
5,19,8.275748
6,20,8.395084
7,21,7.828260
8,23,8.129872
9,29,8.330977


In [180]:
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4], [1, 2], [3, 4],[1, 2], [3, 4]])

y = np.array([1, 2, 3, 4])

kfolds = KFold(n_splits=4, 
                  shuffle=True,
                  random_state=0)

splits = kfolds.split(X)

In [181]:
 [{'train_indexes': test_ix, 'valid_indexes': valid_ix} for (test_ix, valid_ix) in splits]

[{'train_indexes': array([0, 1, 3, 5, 6, 7, 9]),
  'valid_indexes': array([2, 4, 8])},
 {'train_indexes': array([0, 2, 3, 4, 5, 7, 8]),
  'valid_indexes': array([1, 6, 9])},
 {'train_indexes': array([0, 1, 2, 4, 5, 6, 8, 9]),
  'valid_indexes': array([3, 7])},
 {'train_indexes': array([1, 2, 3, 4, 6, 7, 8, 9]),
  'valid_indexes': array([0, 5])}]

In [174]:
 [(test_ix,  valid_ix) for (test_ix, valid_ix) in splits]

[]

In [183]:
list() == []

True

NameError: name 'empty' is not defined

In [241]:
datetime.now().isoformat(' ', 'seconds').replace(':', '-').replace(' ', '-')

TypeError: replace() argument 1 must be str, not tuple

In [242]:
os.getcwd()

'/home/ftam/Documents/data science/kaggel_git/data_science'